In [1]:
import requests 
from config import omdb_key
from config import tmdb_key
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 999)
import time


In [2]:
def tmdb_request(movie_id):
    #movie_id comes from calling 'imdbID' key from the omdb request
    import requests
    url = "https://api.themoviedb.org/3/movie/" + movie_id + "?language=en-US&api_key=" + tmdb_key
    
    #url = "https://api.themoviedb.org/3/movie/tt2527336"

    payload = "{}"
    response = requests.request("GET", url, data=payload)

    return response.json()
    
    
    

In [3]:
def omdb_request(movie_title):
    seperator = '+'
    title_url = seperator.join(movie_title.split())
    url = 'http://www.omdbapi.com/?' + 't=' + title_url + '&apikey=' + omdb_key
    r = requests.get(url)
    return r.json()

In [4]:
def make_int(df_name, column_name):
    empty = []
    
    for index, row in df_name.iterrows():
        if type(row[column_name]) != float:
            plus = row[column_name].split('$')
            dot = plus[-1].split('.')
            number = None

            if len(dot) == 2:
                number = dot[0]+dot[1]
                if number[-1] == 'k':
                    number = number.replace(',','')
                    number = number.replace('k', '00')
                else:
                    number = number.replace(',','') + '00000'
            else:
                
                number = dot[0].replace('k', '000')
                number = number.replace(',','')

            empty.append(int(number))
        else:
            empty.append(int(0))
    df_name[column_name] = empty


In [5]:
def bom_db(year):
    url = 'https://www.boxofficemojo.com/yearly/chart/?view2=worldwide&yr=' + str(year)+'&p=.htm'
    r = pd.read_html(requests.get(url).content)
    
    df = r[2]
    
    df.columns = ['Rank', 'Title', 'Studio', 'Worldwide_Gross', 'Domestic_Gross', 
                   'Domestic%','Overseas_Gross' ,'Overseas%']

    df['Year'] = year
    df.drop([0,1], inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns=['index', 'Rank'], inplace=True)
    
    make_int(df, 'Worldwide_Gross')
    make_int(df, 'Domestic_Gross')
    make_int(df, 'Overseas_Gross')
    return df

In [6]:
def df_with_ratings(year):
    df = bom_db(year) #Creates dataframe using previous function
    #Looping through the title of each movie in the dataframe
    for i in range(0, len(df.Title)):
        try:
            print(df.Title[i])
            title_data = omdb_request(df.Title[i]) #Requests omdb to get data about specific movie
            if title_data['Response'] == 'False':
                df.loc[i, 'rotten_tomatoes_rating'] = np.nan
                df.loc[i, 'imdb_rating'] = np.nan
                df.loc[i, 'meta_score'] = np.nan
                df.loc[i, 'actors'] = np.nan
                df.loc[i, 'rating'] = np.nan
                df.loc[i, 'genre'] = np.nan
                df.loc[i, 'director'] = np.nan
                df.loc[i, 'release_date'] = np.nan
                df.loc[i, 'budget'] = np.nan




            else:
                imdb_score = 0
                rtom_score = 0
                metascore = 0
                if len(title_data['Ratings']) == 3:
                    for rating in title_data['Ratings']:
                        if rating['Source'] == 'Internet Movie Database':
                            imdb_score = float(rating['Value'].split('/')[0])
                        elif rating['Source'] == 'Rotten Tomatoes':

                            rtom_score = float(rating['Value'].split('%')[0])

                        elif rating['Source'] == 'Metacritic':
                            metascore = rating['Value'].split('/')[0]

                    df.loc[i, 'imdb_rating'] = imdb_score
                    df.loc[i, 'rotten_tomatoes_rating'] = rtom_score
                    df.loc[i, 'meta_score'] = metascore
                    df.loc[i, 'actors'] = title_data['Actors']
                    df.loc[i, 'rating'] = title_data['Rated']
                    df.loc[i, 'genre'] = title_data['Genre']
                    df.loc[i, 'director'] = title_data['Director']



                    if len(title_data['Released'].split()) < 2:
                        df.loc[i, 'release_date'] = np.nan
                    else:    

                        df.loc[i, 'release_date'] = title_data['Released'].split()[0] + ' ' + title_data['Released'].split()[1]

                    imdb_id = title_data['imdbID']

                    movie_tmdb_info = tmdb_request(imdb_id)
                    if "status_code" in movie_tmdb_info.keys():
                        df.loc[i, 'budget'] = np.nan
                    else:
                        budget = movie_tmdb_info['budget']
                        df.loc[i, 'budget'] = int(budget)  

                elif len(title_data['Ratings']) == 2:
                    missingid = 0
                    for rating in title_data['Ratings']:
                        if rating['Source'] == 'Internet Movie Database':
                            imdb_score = float(rating['Value'].split('/')[0])
                            missingid = 1

                        elif rating['Source'] == 'Rotten Tomatoes':

                            rtom_score = float(rating['Value'].split('%')[0])
                            missingid = 2

                        elif rating['Source'] == 'Metacritic':
                                metascore = rating['Value'].split('/')[0]
                                missingid = 3
                    if missingid == 2:

                        df.loc[i, 'imdb_rating'] = imdb_score
                        df.loc[i, 'meta_score'] = np.nan
                        df.loc[i, 'rotten_tomatoes_rating'] = rtom_score
                        df.loc[i, 'actors'] = title_data['Actors']
                        df.loc[i, 'rating'] = title_data['Rated']
                        df.loc[i, 'genre'] = title_data['Genre']
                        df.loc[i, 'director'] = title_data['Director']

                        if len(title_data['Released'].split()) < 2:
                            df.loc[i, 'release_date'] = np.nan
                        else:    

                            df.loc[i, 'release_date'] = title_data['Released'].split()[0] + ' ' + title_data['Released'].split()[1]
                        #Calling a different api to get budget 
                        movie_tmdb_info = tmdb_request(imdb_id)
                        if "status_code" in movie_tmdb_info.keys():
                            df.loc[i, 'budget'] = np.nan
                        else:
                            budget = movie_tmdb_info['budget']
                            df.loc[i, 'budget'] = int(budget)


                    elif missingid ==3:
                        df.loc[i, 'imdb_rating'] = imdb_score
                        df.loc[i, 'meta_score'] = metascore
                        df.loc[i, 'rotten_tomatoes_rating'] = np.nan
                        df.loc[i, 'actors'] = title_data['Actors']
                        df.loc[i, 'rating'] = title_data['Rated']
                        df.loc[i, 'genre'] = title_data['Genre']
                        df.loc[i, 'director'] = title_data['Director']

                        if len(title_data['Released'].split()) < 2:
                            df.loc[i, 'release_date'] = np.nan
                        else:    

                            df.loc[i, 'release_date'] = title_data['Released'].split()[0] + ' ' + title_data['Released'].split()[1]
                        #Calling a different api to get budget
                        movie_tmdb_info = tmdb_request(imdb_id)
                        if "status_code" in movie_tmdb_info.keys():
                            df.loc[i, 'budget'] = np.nan
                        else:
                            budget = movie_tmdb_info['budget']
                            df.loc[i, 'budget'] = int(budget)
        
            time.sleep(0.25) 
            print((0.25*len(df.Title))-(i*0.25))
        except:
            pass
    return df   

In [7]:
def super_df(startyear, endyear):
    years = list(range(startyear, endyear+1))
    combined_df = pd.DataFrame()
    
    time = 0
    for year in years:
        new = len(bom_db(year))
        time += (0.25*new)
    print(time)
    
    for year in years:
        temp_df = df_with_ratings(year)
        combined_df = combined_df.append(temp_df)
        print(year)
    combined_df = combined_df.reset_index(drop=True)
    
    return combined_df

In [11]:
df09 = df_with_ratings(2009)

Avatar
86.5
Harry Potter and the Half-Blood Prince
86.25
Ice Age: Dawn of the Dinosaurs
86.0
Transformers: Revenge of the Fallen
85.75
2012
85.5
Up
85.25
The Twilight Saga: New Moon
85.0
Sherlock Holmes
84.75
Angels & Demons
84.5
The Hangover
84.25
Alvin and the Chipmunks: The Squeakquel
84.0
Night at the Museum: Battle of the Smithsonian
83.75
Star Trek
83.5
Monsters Vs. Aliens
83.25
X-Men Origins: Wolverine
83.0
Terminator Salvation
82.75
Fast and Furious
82.5
A Christmas Carol (2009)
82.25
Inglourious Basterds
82.0
The Proposal
81.75
The Blind Side
81.5
G.I. Joe: The Rise of Cobra
81.25
G-Force
81.0
The Princess and the Frog
80.75
Michael Jackson's This Is It
80.5
Cloudy with a Chance of Meatballs
80.25
Taken
80.0
It's Complicated
79.75
Public Enemies
79.5
District 9
79.25
The Ugly Truth
79.0
Ponyo
78.75
Paranormal Activity
78.5
The Final Destination
78.25
Watchmen
78.0
Knowing
77.75
Paul Blart: Mall Cop
77.5
He's Just Not That Into You
77.25
Couples Retreat
77.0
Up in the Air
76.75

In [13]:
df09

,Title,Studio,Worldwide_Gross,Domestic_Gross,Domestic%,Overseas_Gross,Overseas%,Year,imdb_rating,rotten_tomatoes_rating,meta_score,actors,rating,genre,director,release_date,budget
0,Avatar,Fox,2777200000,749800000,27.0%,2027500000,73.0%,2009,7.8,82.0,83,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",PG-13,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,18 Dec,237000000.0
1,Harry Potter and the Half-Blood Prince,WB,934400000,302000000,32.3%,632500000,67.7%,2009,7.6,83.0,78,"Daniel Radcliffe, Michael Gambon, Dave Legeno,...",PG,"Adventure, Family, Fantasy, Mystery",David Yates,15 Jul,250000000.0
2,Ice Age: Dawn of the Dinosaurs,Fox,886700000,196600000,22.2%,690100000,77.8%,2009,7.0,46.0,50,"Eunice Cho, Karen Disher, Harrison Fahn, Maile...",PG,"Animation, Action, Adventure, Comedy, Family","Carlos Saldanha, Mike Thurmeier(co-director)",01 Jul,90000000.0
3,Transformers: Revenge of the Fallen,P/DW,836300000,402100000,48.1%,434200000,51.9%,2009,6.0,19.0,35,"Shia LaBeouf, Megan Fox, Josh Duhamel, Tyrese ...",PG-13,"Action, Adventure, Sci-Fi",Michael Bay,24 Jun,150000000.0
4,2012,Sony,769700000,166100000,21.6%,603600000,78.4%,2009,5.8,40.0,49,"John Cusack, Amanda Peet, Chiwetel Ejiofor, Th...",PG-13,"Action, Adventure, Sci-Fi",Roland Emmerich,13 Nov,200000000.0
5,Up,BV,735100000,293000000,39.9%,442100000,60.1%,2009,8.3,98.0,88,"Edward Asner, Christopher Plummer, Jordan Naga...",PG,"Animation, Adventure, Comedy, Family","Pete Docter, Bob Peterson(co-director)",29 May,175000000.0
6,The Twilight Saga: New Moon,Sum.,709700000,296600000,41.8%,413100000,58.2%,2009,4.7,28.0,44,"Kristen Stewart, Christina Jastrzembska, Rober...",PG-13,"Adventure, Drama, Fantasy, Romance",Chris Weitz,20 Nov,50000000.0
7,Sherlock Holmes,WB,524000000,209000000,39.9%,315000000,60.1%,2009,7.6,70.0,57,"Robert Downey Jr., Jude Law, Rachel McAdams, M...",PG-13,"Action, Adventure, Crime, Mystery, Thriller",Guy Ritchie,25 Dec,90000000.0
8,Angels & Demons,Sony,485900000,133400000,27.4%,352600000,72.6%,2009,6.7,37.0,48,"Tom Hanks, Ewan McGregor, Ayelet Zurer, Stella...",PG-13,"Mystery, Thriller",Ron Howard,15 May,150000000.0
9,The Hangover,WB,467500000,277300000,59.3%,190200000,40.7%,2009,7.7,78.0,73,"Bradley Cooper, Ed Helms, Zach Galifianakis, J...",R,Comedy,Todd Phillips,05 Jun,35000000.0


In [14]:
df09.to_csv('df09')

In [15]:
df_last = super_df(2010, 2019)

859.0
Toy Story 3
82.0
Alice in Wonderland (2010)
81.75
Harry Potter and the Deathly Hallows Part 1
81.5
Inception
81.25
Shrek Forever After
81.0
The Twilight Saga: Eclipse
80.75
Iron Man 2
80.5
Tangled
80.25
Despicable Me
80.0
How to Train Your Dragon
79.75
Clash of the Titans (2010)
79.5
The Chronicles of Narnia: The Voyage of the Dawn Treader
79.25
The King's Speech
79.0
Tron Legacy
78.75
The Karate Kid
78.5
Prince of Persia: The Sands of Time
78.25
Black Swan
78.0
Megamind
77.75
Robin Hood
77.5
The Last Airbender
77.25
Little Fockers
77.0
Resident Evil: Afterlife
76.75
Shutter Island
76.5
Salt
76.25
Sex and the City 2
76.0
The Tourist
75.75
The Expendables
75.5
Grown Ups
75.25
Knight & Day
75.0
True Grit
74.75
Gulliver's Travels
74.5
Percy Jackson & The Olympians: The Lightning Thief
74.25
The Social Network
74.0
Valentine's Day
73.75
The Sorcerer's Apprentice
73.5
Due Date
73.25
Eat Pray Love
73.0
Yogi Bear
72.75
Red
72.5
Paranormal Activity 2
72.25
The A-Team
72.0
Jackass 3-D
71.

89.75
In Time
89.5
Zookeeper
89.25
The Muppets
89.0
Limitless
88.75
Johnny English Reborn
88.5
Final Destination 5
88.25
Tower Heist
88.0
Midnight in Paris
87.75
Happy Feet Two
87.5
I Am Number Four
87.25
Jack and Jill
87.0
Friends with Benefits
86.75
No Strings Attached
86.5
Arthur Christmas
86.25
Source Code
86.0
Crazy, Stupid, Love.
85.75
New Year's Eve
85.5
Contagion
85.25
The Artist
85.0
The Three Musketeers (2011)
84.75
Unknown
84.5
The Adjustment Bureau
84.25
We Bought a Zoo
84.0
Water for Elephants
83.75
The Iron Lady
83.5
Moneyball
83.25
Sanctum
83.0
Justin Bieber: Never Say Never
82.75
Paul
82.5
Scream 4
82.25
Insidious
82.0
The Rite
81.75
Dolphin Tale
81.5
Season of the Witch
81.25
Sucker Punch
81.0
Red Riding Hood
80.75
Spy Kids: All the Time in the World
80.5
J. Edgar
80.25
Hall Pass
80.0
Big Mommas: Like Father, Like Son
79.75
Abduction
79.5
Tinker, Tailor, Soldier, Spy
79.25
Priest
79.0
Drive (2011)
78.75
The Mechanic
78.5
The Ides of March
78.25
The Change-Up
78.0
The L

98.5
The Amazing Spider-Man
98.25
Madagascar 3: Europe's Most Wanted
98.0
The Hunger Games
97.75
MIB 3
97.5
Life of Pi
97.25
Ted
97.0
Brave
96.75
Wreck-It Ralph
96.5
Les Miserables (2012)
96.25
The Intouchables (U.S.-only)
96.0
Django Unchained
95.75
Prometheus
95.5
Snow White and the Huntsman
95.25
Taken 2
95.0
Hotel Transylvania
94.75
Dr. Seuss' The Lorax
94.5
Titanic 3D
94.25
Journey 2: The Mysterious Island
94.0
The Expendables 2
93.75
Rise of the Guardians
93.5
Wrath of the Titans
93.25
Battleship
93.0
John Carter
92.75
The Bourne Legacy
92.5
Lincoln
92.25
Dark Shadows
92.0
Resident Evil: Retribution
91.75
Silver Linings Playbook
91.5
American Reunion
91.25
Argo
91.0
Jack Reacher
90.75
Safe House
90.5
21 Jump Street
90.25
Total Recall (2012)
90.0
The Vow
89.75
Mirror Mirror
89.5
The Impossible
89.25
The Dictator
89.0
Looper
88.75
Magic Mike
88.5
Flight
88.25
Underworld Awakening
88.0
This Means War
87.75
The Secret World of Arrietty
87.5
Paranormal Activity 4
87.25
Step Up Revolut

6.5
Juan of the Dead
6.25
4:44: Last Day on Earth
6.0
Little Birds
5.75
Return (2012)
5.5
Alps
5.25
Patience (After Sebald)
5.0
Take Me Home
4.75
Reuniting the Rubins
4.5
Cirkus Columbia
4.25
Grassroots
4.0
Husbands in Goa
3.75
Starry Starry Night
3.5
Corpo Celeste
3.25
Dreams of a Life
3.0
The Babymakers
2.75
Las Acacias
2.5
The Good Doctor
2.25
Liar's Autobiography
2.0
Sleepless Night
1.75
ATM
1.5
About Cherry
1.25
Perfect Sense
1.0
Cheerful Weather for the Wedding
0.75
The Decoy Bride
0.5
Death of a Superhero
0.25
2012


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


...

0.75
Open Windows
0.5
Mr. Pip
0.25
2014
Star Wars: The Force Awakens
112.5
Jurassic World
112.25
Furious 7
112.0
Avengers: Age of Ultron
111.75
Minions
111.5
Spectre
111.25
Inside Out
111.0
Mission: Impossible - Rogue Nation
110.75
The Hunger Games: Mockingjay - Part 2
110.5
The Martian
110.25
Fifty Shades of Grey
110.0
Cinderella (2015)
109.75
The Revenant
109.5
Ant-Man
109.25
Hotel Transylvania 2
109.0
San Andreas
108.75
Terminator: Genisys
108.5
Kingsman: The Secret Service
108.25
Home (2015)
108.0
Mad Max: Fury Road
107.75
The Good Dinosaur
107.5
Taken 3
107.25
The SpongeBob Movie: Sponge Out of Water
107.0
Maze Runner: The Scorch Trials
106.75
The Divergent Series: Insurgent
106.5
Pitch Perfect 2
106.25
Paddington
106.0
Mojin: The Lost Legend
105.75
Lost in Hong Kong
105.5
The Peanuts Movie
105.25
Pixels
105.0
Daddy's Home
104.75
Spy
104.5
Alvin and the Chipmunks: The Road Chip
104.25
Goodbye Mr. Loser
104.0
Ted 2
103.75
Tomorrowland
103.5
Everest (2015)
103.25
Straight Outta Comp

22.0
Love Thy Nature
21.75
The Advocate: A Missing Body
21.5
Heist (2015)
21.25
Difret
21.0
Appropriate Behavior
20.75
Beyond the Reach
20.5
The Salvation
20.25
Madame Bovary
20.0
Charlie's Country
19.75
Pearl Button
19.5
Queen and Country
19.25
The Phone
19.0
What Our Fathers Did: A Nazi Legacy
18.75
Lila & Eve
18.5
Before We Go
18.25
The Mafia Only Kills in Summer
18.0
Knock Knock
17.75
Kindergarten Teacher
17.5
Welcome to Leith
17.25
Wild City
17.0
The Forbidden Room
16.75
Song One
16.5
Love Forecast
16.25
Out 1, noli me tangere
16.0
3 1/2 Minutes, 10 Bullets
15.75
Paulette
15.5
The Cut
15.25
The Keeping Room
15.0
Big Game
14.75
Kill Me Three Times
14.5
Hippocrates, Diary of a French Doctor
14.25
Court
14.0
Salut D'Amour
13.75
Mediterranea
13.5
Noma - My Perfect Storm
13.25
The Little Death
13.0
Gangnam 1970
12.75
Strangerland
12.5
Eastern Boys
12.25
Extraction
12.0
Eva
11.75
The World of Kanako
11.5
Zombeavers
11.25
The Hallow
11.0
Amour Fou
10.75
Arabian Nights: Volume 1 - The Res

41.0
Tunnel
40.75
Knight of Cups
40.5
Harry & Snowman
40.25
Viral
40.0
American Pastoral
39.75
The Age of Shadows
39.5
Fitoor
39.25
The First Monday in May
39.0
Marguerite
38.75
The Boy and the Beast
38.5
Wiener-Dog
38.25
Our Little Sister
38.0
Only Yesterday (2016 re-release)
37.75
Baaghi
37.5
Land of Mine
37.25
Things To Come
37.0
The Lobster (Canada Release)
36.75
High-Rise
36.5
Te3n
36.25
Francofonia
36.0
Mia Madre
35.75
April and the Extraordinary World
35.5
Ixcanul
35.25
The People vs. Fritz Bauer
35.0
Aquarius
34.75
Kingsglaive: Final Fantasy: XV
34.5
My Golden Days
34.25
The Family Fang
34.0
Dheepan
33.75
Life, Animated
33.5
Oasis: Supersonic
33.25
The Invitation
33.0
Miss Hokusai
32.75
Kaptaan
32.5
League of Gods
32.25
Viva
32.0
The Wave
31.75
Asura: The City of Madness
31.5
Louder than Bombs
31.25
Howards End (2016 Restoration)
31.0
Sunset Song
30.75
Born to be Blue
30.5
Seasons
30.25
Rams (Hrútar)
30.0
Tumbledown
29.75
The Brand New Testament
29.5
Spirits' Homecoming
29.25
S

31.5
Poster Boys
31.25
The Salesman
31.0
Pokemon the Movie: I Choose You!
30.75
Jeepers Creepers 3
30.5
Un Padre No Tan Padre
30.25
Good Time
30.0
Everybody Loves Somebody
29.75
Along with the Gods: The Two Worlds
29.5
Youth (2017)
29.25
Last Flag Flying
29.0
A Quiet Passion
28.75
Free Fire
28.5
The Last Word
28.25
Human Flow
28.0
Jane (2017)
27.75
A Ghost Story
27.5
The Square
27.25
Patti Cake$
27.0
The Wedding Plan
26.75
Personal Shopper
26.5
Churchill
26.25
The Sense of an Ending
26.0
3 Idiotas
25.75
6 Below: Miracle on the Mountain
25.5
Lady Macbeth
25.25
Film Stars Don't Die in Liverpool
25.0
Yu-Gi-Oh! The Dark Side of Dimensions
24.75
Lucky (2017)
24.5
Faces Places
24.25
Tubelight
24.0
Frantz
23.75
BOMBSHELL: The Hedy Lamarr Story
23.5
Walk With Me
23.25
Buena Vista Social Club: Adios
23.0
Titanic (20th Anniversary)
22.75
Lost in Paris
22.5
Shubh Mangal Saavdhan
22.25
The Midwife
22.0
3 Generations
21.75
Fairy Tail: Dragon Cry
21.5
Our President
21.25
Letters from Baghdad
21.0
Br

16.0
Museo
15.75
The Little Mermaid (2018)
15.5
Ghost Stories
15.25
The Young Karl Marx
15.0
Becoming Astrid
14.75
Ryuichi Sakamoto: Coda
14.5
Gonjiam: Haunted Asylum
14.25
Take Point
14.0
The Dawn Wall
13.75
The Negotiation
13.5
The Captain
13.25
In Between
13.0
Namaste England
12.75
Happy Phirr Bhag Jayegi
12.5
Ismael's Ghosts
12.25
Revenge (2018)
12.0
Unstoppable (2018)
11.75
Reign of Judges: Title of Liberty - Concept Short
11.5
Mary Shelley
11.25
The Third Murder
11.0
The House That Jack Built
10.75
Custody
10.5
Godard Mon Amour
10.25
Just a Breath Away
10.0
Mukkabaaz
9.75
Keys to the Heart
9.5
Tehran Taboo
9.25
The Last Suit
9.0
Nico, 1988
8.75
Antonio Lopez 1970: Sex Fashion & Disco
8.5
Helicopter Eela
8.25
Under the Tree
8.0
Before We Vanish
7.75
I Am Not a Witch
7.5
Racer & the Jailbird
7.25
Rodin
7.0
What Will People Say
6.75
Oolong Courtyard
6.5
Girls vs Gangsters
6.25
No Date, No Signature
6.0
A Ciambra
5.75
The Apparition (2018)
5.5
The Charmer
5.25
Elliot: The Littlest Re

In [16]:
df_last

,Domestic%,Domestic_Gross,Overseas%,Overseas_Gross,Studio,Title,Worldwide_Gross,Year,actors,budget,director,genre,imdb_rating,meta_score,rating,release_date,rotten_tomatoes_rating
0,38.9%,415000000,61.1%,652000000,BV,Toy Story 3,1067000000,2010,"Tom Hanks, Tim Allen, Joan Cusack, Ned Beatty",200000000.0,Lee Unkrich,"Animation, Adventure, Comedy, Family, Fantasy",8.3,92,G,18 Jun,98.0
1,32.6%,334200000,67.4%,691300000,BV,Alice in Wonderland (2010),1025500000,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30.8%,296000000,69.2%,664300000,WB,Harry Potter and the Deathly Hallows Part 1,960300000,2010,"Bill Nighy, Emma Watson, Richard Griffiths, Ha...",250000000.0,David Yates,"Adventure, Fantasy, Mystery",7.7,65,PG-13,19 Nov,78.0
3,35.3%,292600000,64.7%,535700000,WB,Inception,828300000,2010,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...",160000000.0,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",8.8,74,PG-13,16 Jul,86.0
4,31.7%,238700000,68.3%,513900000,P/DW,Shrek Forever After,752600000,2010,"Mike Myers, Eddie Murphy, Cameron Diaz, Antoni...",165000000.0,Mike Mitchell,"Animation, Adventure, Comedy, Family, Fantasy,...",6.3,58,PG,21 May,58.0
5,43.0%,300500000,57.0%,398000000,Sum.,The Twilight Saga: Eclipse,698500000,2010,"Xavier Samuel, Kristen Stewart, Robert Pattins...",68000000.0,David Slade,"Adventure, Drama, Fantasy, Romance",5.0,58,PG-13,30 Jun,48.0
6,50.1%,312400000,49.9%,311500000,Par.,Iron Man 2,623900000,2010,"Robert Downey Jr., Gwyneth Paltrow, Don Cheadl...",200000000.0,Jon Favreau,"Action, Adventure, Sci-Fi",7.0,57,PG-13,07 May,73.0
7,33.9%,200800000,66.1%,391000000,BV,Tangled,591800000,2010,"Mandy Moore, Zachary Levi, Donna Murphy, Ron P...",260000000.0,"Nathan Greno, Byron Howard","Animation, Adventure, Comedy, Family, Fantasy,...",7.8,71,PG,24 Nov,89.0
8,46.3%,251500000,53.7%,291600000,Uni.,Despicable Me,543100000,2010,"Steve Carell, Jason Segel, Russell Brand, Juli...",69000000.0,"Pierre Coffin, Chris Renaud","Animation, Comedy, Family, Fantasy",7.7,72,PG,09 Jul,81.0
9,44.0%,217600000,56.0%,277300000,P/DW,How to Train Your Dragon,494900000,2010,"Jay Baruchel, Gerard Butler, Craig Ferguson, A...",165000000.0,"Dean DeBlois, Chris Sanders","Animation, Action, Adventure, Family, Fantasy",8.1,74,PG,26 Mar,98.0


In [17]:
df_last.to_csv('dflast')